<a href="https://colab.research.google.com/github/MarosBenadik/CHatingRoomsApp/blob/dev/Disertatio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [541]:
# IMPORTS
from spacy.lang.en.stop_words import STOP_WORDS

from nltk.stem.porter import PorterStemmer

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import decomposition

from collections import Counter
from collections import OrderedDict

from operator import itemgetter

import numpy as np
import pandas as pd
import geopy.distance

import requests
import json
import random
import nltk
import difflib
import spacy
import re, string

# This tokenizer divides a text into a list of sentences by using
# an unsupervised algorithm to build a model for abbreviation words, collocations, and words.
nltk.download('punkt')

pd.options.mode.chained_assignment = None

print('OK')

OK


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [542]:
# Check the version of Libraries
print(f"Pandas version: {pd.__version__}")
print(f"Numpy version: {np.__version__}")

Pandas version: 1.3.5
Numpy version: 1.21.6


In [543]:
# Main Tags 

tags = [
	"skin", "haircut", "makeup", "colouring", "color", "colour", "yoga", 
	"nutrition", "fitness", "tanning", "piercing", "tattoo", "aesthetics", "spa",
	"barber", "eyelash", "eyelashes", "eyebrow", "eyebrows", "wellness", "body",
	"massage", "hair", "nail", "nails", "face", "product", "health", "hand", "general"
]

In [544]:
# Request to the server to obtain Artist Data
ds = pd.read_json('/content/Artist_200.json')


# Adding required collumns to dataset 
ds.insert(0, 'activity', 0)
ds.insert(0, 'price', 0)


In [545]:
# Function to get Artist Price

def getPrice(artist):
  if artist['price_range']['lowest'] is None:
    return random.randint(10,150)
  else:
    return (int(artist['price_range']['highest']) + int(artist['price_range']['lowest'])) / 2
  
  return 

# Curently Generating Random number to simulate Artist activity on the platform

for index, artist in ds.iterrows():

  ds['longitude'][index] = random.uniform(2, -4)
  ds['latitude'][index] = random.uniform(48, 52)
  ds['price'][index] = getPrice(artist)
  if not artist['collection_tags']:
    ds['collection_tags'][index] = random.choices(tags, k=4)
  ds['activity'][index] = random.randint(0,7)
  if ds['occupation'][index] is None:
    ds['occupation'][index] = "beautician"

selectedFeatures = ['user_id', 'collection_tags', 'longitude', 'latitude',
                    'occupation', 'activity', 'price']

#for feature in selectedFeatures:
#  print(feature)
#  ds[feature] = ds[feature].fillna(0)

combined_Feature = [ds['user_id'],
                    ds['latitude'].apply(str),
                    ds['longitude'].apply(str),
                    ds['occupation'].apply(str),
                    ds['collection_tags'],
                    ds['activity'],
                    ds['price']]

artist_data = pd.DataFrame(combined_Feature).transpose()

print('OK')


OK


In [546]:
# Artist Data visualization after Pre-Procesing
artist_data.head()

,user_id,latitude,longitude,occupation,collection_tags,activity,price
0,ba64dbbd-bfa6-4d8f-bf41-ee832f0de4fd,50.84596662625389,-3.553249732940091,Speech Pathologist,"[eyelashes, aesthetics, general, health]",5,129
1,f78970f6-0d0f-4717-97f3-f8886a8201c9,50.911014152714,-0.8962816903064645,Operator,"[skin, eyelash, massage, fitness]",3,101
2,a8a6aa5f-d774-435b-b006-19c5c211b86b,50.97746145802806,-3.8769603429192188,Nail Professional,"[Nails, Face, Hair, Hair Removal, Body, Massag...",0,34
3,a51d1e71-d715-4772-b27a-f7f804de948a,51.577095533662046,1.4539147300329702,Makeup Artist,"[Face, Nails, Hair, Hair Removal, Body, Massag...",7,74
4,e1302c2c-52e8-413c-8681-3094d691f150,50.31117328293671,-3.5546261465082187,Staff Scientist,"[makeup, tanning, massage, color]",0,69


In [547]:
# Stemmer Activation 
stemmer = PorterStemmer()
nlp = spacy.load("en_core_web_sm")

In [548]:
# actual request to the server to get questions data using credentials

questions = pd.read_json("/content/Questions_600.json")

questions.insert(0, 'longitude', 0.0)
questions.insert(0, 'latitude', 0.0)
questions.insert(0, 'user_id', 0)
questions.insert(0, 'occupation', 0)
questions.insert(0, 'category', "")
questions.insert(0, 'distance', 0.0)
questions.insert(0, 'best_artist', "")
print("OK")

OK


In [549]:
# Function to obtain user location
# As these are dummy data the random numbers are being generated to simulate londons 
# location 

def getUserOccupation(userId):
  occupation = random.choice(list(ds['occupation']))
  return occupation

# Function to obtain tokenized data

def tokenize(text):
  tokens = [word for word in nltk.word_tokenize(text) if (len(word) > 3 and len(word.strip('Xx/')) > 2)]
  return tokens

# DATA Preprocesing 

for index, question in questions.iterrows():
  questions['user_id'][index] = questions['user_details'][index]['user_id']
  questions['occupation'][index] = getUserOccupation("userId")
  questions['longitude'][index] = round(random.uniform(1, -4), 8)
  questions['latitude'][index] = round(random.uniform(48, 52), 8)

  if not question['question_description']:
    questions['question_description'][index] = " "
  elif question['question_description'] is None:
    questions['question_description'][index] = " "
  else:
    pass

  questionContent = question['question_header'] + ' , ' + str(question['question_description'])

  chars = '[%s]+' % re.escape(string.punctuation)

  questionContent = re.sub(chars, '', questionContent)

  doc = nlp(questionContent)

  tokenized_q_content = []

  for token in doc:
    tokenized_q_content.append(token.lemma_.lower())

  for word in tokenized_q_content:
    if word in STOP_WORDS:
      tokenized_q_content.remove(word)
    if word == "  ":
      tokenized_q_content.remove(word)

  for word in tokenized_q_content:
    if word in tags:
      questions['category'][index] = word
      break
    else:
      questions['category'][index] = "general"

train, test = [], []

for index, question in questions.iterrows():
  if question['category'] != "general":
    train.append(question)
  else:
    test.append(question)

train = pd.DataFrame.from_records(train)
test = pd.DataFrame.from_records(test)

train['question_content'] = train['question_header'] + ',' + train['question_description']

train_df = train[['question_content', 'question_id', 'anonymous', 'category', 'user_id', 'occupation', 'longitude', 'latitude', 'distance', 'best_artist']]

test['question_content'] = test['question_header'] + ',' + test['question_description']

test_df = test[['question_content', 'question_id', 'anonymous', 'user_id', 'occupation', 'longitude', 'latitude', 'distance', 'best_artist']]

vectorizer_tf = TfidfVectorizer(tokenizer=tokenize, stop_words='english', max_df=0.75, max_features=10000, use_idf=False, norm=None)

tf_vectors = vectorizer_tf.fit_transform(train_df.question_content)

lda = decomposition.LatentDirichletAllocation(n_components=30, max_iter=5, learning_method='online', learning_offset=50, n_jobs=-1, random_state=2)

W1 = lda.fit_transform(tf_vectors)
H1 = lda.components_

num_words = 6

vocab = np.array(vectorizer_tf.get_feature_names_out())

top_words = lambda t: [vocab[i] for i in np.argsort(t)[:-num_words-1:-1]]

topic_word = ([top_words(t) for t in H1])

topics = [' '.join(t) for t in topic_word]

colnames = [tags[i] for i in range(lda.n_components)]
docnames = ["Question " + str(i) for i in range(len(train_df.question_content))]
df_doc_topic = pd.DataFrame(np.round(W1, 2), columns=colnames, index=docnames)
significant_topic = np.argmax(df_doc_topic.values, axis=1)
df_doc_topic['dominant_topic'] = significant_topic

WTest = lda.transform(vectorizer_tf.transform(test_df.question_content))

colnames = [tags[i] for i in range(lda.n_components)]
docnames = ["Question " + str(i) for i in range(len(test_df.question_content))]
df_doc_topic_test = pd.DataFrame(np.round(WTest, 2), columns=colnames, index=docnames)
significant_topic = np.argmax(df_doc_topic_test.values, axis=1)
df_doc_topic_test['dominant_topic'] = significant_topic
  

  #for index, value in enumerate(df_doc_topic_test['dominant_topic'].items()):
    #print(f"question: {test_df['question_content'].values[int(index)]}, category: {tags[value[1]]}")

  #print(f"test data: {len(test_df)}, number of topics: {len(df_doc_topic_test)}")

clasifiedData = []

for index, row in test_df.iterrows():
  
  category = tags[df_doc_topic_test['dominant_topic'][index]]
  row['category'] = category
  clasifiedData.append(row)

clasifiedData = pd.DataFrame(clasifiedData)

frames = [train_df, clasifiedData]

questionsResult = pd.concat(frames, ignore_index=True)

print("ok")

ok


In [550]:
questionsResult.head()

,question_content,question_id,anonymous,category,user_id,occupation,longitude,latitude,distance,best_artist
0,"Like your skin, do you feel it is necessary bo...",4a02f3b5-d5e4-48ab-9a01-ff3593ff39a3,False,skin,7d3b9c73-df51-49a4-bfbb-ba2950a46495,Makeup Artist,-2.572148,50.393156,0.0,
1,Can I just have my massage in the nude and not...,8a519350-0e8f-409e-a026-58f0c5bb9e43,False,massage,11e6bf27-c18f-4f6b-9664-8a2b86b33292,GIS Technical Architect,-1.207895,48.641239,0.0,
2,Have you ever experienced any allergic reactio...,335623e5-956f-4ce6-99c3-55b635bd54f6,False,product,c0732824-a076-428a-9898-ab67cbac1172,Barber,-1.100071,48.991784,0.0,
3,Does it affect the dye job if my colouring hai...,4c6dbee4-7f69-4a92-a2a4-c03d4bf054b8,False,colouring,f508c422-ba39-4cd3-ac8e-d9a569936c92,Colorist,-1.965606,49.793537,0.0,
4,DO GEL NAILS HELP IF YOU ARE A NAIL-BITER AND/...,ac75d1ea-66e3-4ca7-be22-c9b6b0a3edb3,False,nails,1a07bd99-b7fd-4a29-b651-231f61868c84,VP Accounting,-1.743845,49.234123,0.0,


In [551]:
def getBestArtist(question, data):
  # List to store filtered data
  artistLocationMatches = []
  artistLocationFiltered = []
  artistTagsMatches = []
  artistPriceMatches = []

  # Function which calculates the distance between user and artist

  def getDistance(userLatitude, userLongitude, artistLatitude, artistLongitude):
    return geopy.distance.geodesic((userLatitude, userLongitude), (artistLatitude, artistLongitude)).km

  # Filter artist based on the tag maching the question category

  for index, artist in data.iterrows():
    for i in range(len(artist['collection_tags'])):
      artist['collection_tags'][i] = artist['collection_tags'][i].lower()

    if question['category'] in artist['collection_tags']:
      artistTagsMatches.append(artist)

  # storing user data with calulated distance in the list

  for index, artist in enumerate(artistTagsMatches):
    distance = getDistance(question['latitude'], question['longitude'], artist['latitude'], artist['longitude'])
    userIndex = np.where(data['user_id'] == artist['user_id'])
    artistLocationMatches.append([distance, userIndex[0][0]])

  # If location is in radius of 100 km artist will be moved to futher filtering

  SortedArtistLocation = sorted(artistLocationMatches,
                                key=itemgetter(0))   

  SortedArtistLocation = SortedArtistLocation[:10]

  # Futher filtering to find 3 artist with best price range

  for artist in SortedArtistLocation:
    artistPriceMatches.append([data['price'][artist[1]], artist[1]])

  PriceMatches = sorted(artistPriceMatches,
                                key=itemgetter(0))

  PriceMatches = PriceMatches[-3:]

  # Chousing the most active artist

  ArtistActivity = []

  for artist in PriceMatches:
    ArtistActivity.append([data['activity'][artist[1]], artist[1]])

  MostActive = sorted(ArtistActivity,
                      reverse=True,
                      key=itemgetter(0))
    
  response = data['user_id'][MostActive[0][1]]

  return response

for index, question in questionsResult.iterrows():
  questionsResult.at[index, 'best_artist'] = getBestArtist(question, artist_data)

selectedFeatures = ['question_id', 'user_id', 'question_header ', 'question_description',
                    'occupation', 'activity', 'best_artist', 'location.longitude', 'location.latitude']

questions_data = pd.DataFrame(questionsResult)

print("ok")

ok


*italicised text*

In [552]:
# Visualisation of Question data after pre-procesing
questions_data.head()




,question_content,question_id,anonymous,category,user_id,occupation,longitude,latitude,distance,best_artist
0,"Like your skin, do you feel it is necessary bo...",4a02f3b5-d5e4-48ab-9a01-ff3593ff39a3,False,skin,7d3b9c73-df51-49a4-bfbb-ba2950a46495,Makeup Artist,-2.572148,50.393156,0.0,f78970f6-0d0f-4717-97f3-f8886a8201c9
1,Can I just have my massage in the nude and not...,8a519350-0e8f-409e-a026-58f0c5bb9e43,False,massage,11e6bf27-c18f-4f6b-9664-8a2b86b33292,GIS Technical Architect,-1.207895,48.641239,0.0,d32d89cd-6b79-47ff-8547-3d89db66c685
2,Have you ever experienced any allergic reactio...,335623e5-956f-4ce6-99c3-55b635bd54f6,False,product,c0732824-a076-428a-9898-ab67cbac1172,Barber,-1.100071,48.991784,0.0,5862c052-069b-40ca-92ec-0f0acdc5c24d
3,Does it affect the dye job if my colouring hai...,4c6dbee4-7f69-4a92-a2a4-c03d4bf054b8,False,colouring,f508c422-ba39-4cd3-ac8e-d9a569936c92,Colorist,-1.965606,49.793537,0.0,3fda8544-92fb-40a2-8f76-aa7170a7b910
4,DO GEL NAILS HELP IF YOU ARE A NAIL-BITER AND/...,ac75d1ea-66e3-4ca7-be22-c9b6b0a3edb3,False,nails,1a07bd99-b7fd-4a29-b651-231f61868c84,VP Accounting,-1.743845,49.234123,0.0,34a0b6d4-847b-4ca2-a9d9-60d48edeb998


In [553]:
vectorizer = TfidfVectorizer()

artist_d = artist_data

X = questions_data.drop('best_artist', axis=1)
y = questions_data['best_artist']

X = (X['question_id'].astype("string") +
     X['category'].astype("string") +
     X['user_id'].astype("string") +
     X['longitude'].astype("string") +
     X['latitude'].astype("string") +
     X['question_content'].astype("string")
    )

X = vectorizer.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
                                            X,
                                            y,
                                            test_size=0.2,
                                            random_state=48
                                            )

print('Ok')

Ok


In [554]:
#Import Random Forest Model
from sklearn.ensemble import RandomForestClassifier

#Create a Gaussian Classifier
clf=RandomForestClassifier(n_estimators=80)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)

y_pred=clf.predict(X_test)

In [555]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.3333333333333333


In [556]:
V_artist = artist_data['user_id'] + " , " + artist_data['latitude'] + " , " + artist_data['longitude'] + " , " + artist_data['occupation'] + " , " + artist_data['activity'].astype('string') + " , " + artist_data['price'].astype('string')

artist_vectors = vectorizer.fit_transform(V_artist)

similarity = cosine_similarity(artist_vectors)

list_of_all_artist = artist_data['user_id'].tolist()

y = 1

for prediction in y_pred:
  find_close_match = difflib.get_close_matches(prediction, list_of_all_artist)
  close_match = find_close_match[0]
  index_of_the_artist = artist_data[artist_data.user_id == close_match]

  similarity_score = list(enumerate(similarity[index_of_the_artist.index.values[0]]))

  sorted_similar_artist = sorted(similarity_score, key = lambda x:x[1], reverse = True) 
  print('\n Suggested Artist for question: ' , y, '\n')

  i = 1
  y+=1

  for artist in sorted_similar_artist:
    index = artist[0]
    id_from_index = artist_d[artist_d.index==index]['user_id'].values[0]
    if (i<4):
      print(i, '.',id_from_index)
      i+=1





 Suggested Artist for question:  1 

1 . 6da7866a-7204-47e9-bf2a-fddf3c81349a
2 . f4ca3137-8400-4bca-aef5-e9247e28800e
3 . ca2d95bf-916a-4d19-b9f5-b90166266519

 Suggested Artist for question:  2 

1 . 5862c052-069b-40ca-92ec-0f0acdc5c24d
2 . 1ad2891e-c394-4bd5-99fe-3014beb37fe9
3 . 29466f3e-41c7-4b10-92ec-5bb0ae6b89ad

 Suggested Artist for question:  3 

1 . 6da7866a-7204-47e9-bf2a-fddf3c81349a
2 . f4ca3137-8400-4bca-aef5-e9247e28800e
3 . ca2d95bf-916a-4d19-b9f5-b90166266519

 Suggested Artist for question:  4 

1 . f78970f6-0d0f-4717-97f3-f8886a8201c9
2 . fd7066b2-d324-4f41-b69f-b14cedacbfa0
3 . decada32-6347-4839-ba2c-37f83dbc9eb0

 Suggested Artist for question:  5 

1 . f78970f6-0d0f-4717-97f3-f8886a8201c9
2 . fd7066b2-d324-4f41-b69f-b14cedacbfa0
3 . decada32-6347-4839-ba2c-37f83dbc9eb0

 Suggested Artist for question:  6 

1 . f78970f6-0d0f-4717-97f3-f8886a8201c9
2 . fd7066b2-d324-4f41-b69f-b14cedacbfa0
3 . decada32-6347-4839-ba2c-37f83dbc9eb0

 Suggested Artist for question:  